In [7]:
import numpy as np
import pandas as pd
import datetime as dt
import altair as alt
alt.data_transformers.enable("json")

DataTransformerRegistry.enable('json')

# Open data

Radiosonde synoptic winds

In [ ]:
df1 = pd.read_parquet("../sail/synoptic_winds_500_local.parquet").assign(pressure = 500)
df2 = pd.read_parquet("../sail/synoptic_winds_700_local.parquet").assign(pressure = 700)
df1.time = df1.time.apply(lambda dt: dt.replace(minute=0, second=0))
df2.time = df2.time.apply(lambda dt: dt.replace(minute=0, second=0))
synopticwinds_df = pd.concat([df1, df2])
synopticwinds_df = synopticwinds_df.groupby(['time', 'pressure']).mean().reset_index()

In [ ]:
alt.Chart(
    synopticwinds_df
).mark_bar().encode(
    alt.X("wspd:Q").bin(True, maxbins=30),
    alt.Y("count():Q")
)

Doppler Lidar (aggregated) data

In [9]:
dl_df = pd.read_parquet("/Users/elischwat/Development/data/sublimationofsnow/sail_processed/gucdlrhiM1.b1/").set_index([
    'z_binned',	'x_offset',	'scan_time'
])['streamwise_velocity'].reset_index()

dl_df = dl_df[dl_df.scan_time >= "2023-02-09 0000"]

# Plot vertical profiles over a day

In [10]:
src = dl_df[dl_df.scan_time.dt.date == dt.date(2023,4,16)].query("x_offset == -250")
src['hour'] = src.scan_time.dt.hour
src['minute'] = src.scan_time.dt.minute
src = src[src.z_binned < 800]
alt.Chart(src).mark_line().encode(
    alt.X("median:Q").sort('-y').title("Downvalley wind (m/s)"),
    alt.Y("z_binned:Q").title("Height (m, agl)"),
    alt.Color("minute:O").scale(scheme='turbo'),
    alt.Facet("hour", columns=6)
).properties(width = 125, height = 125, title = f"Along-valley wind profiles on {str(src.scan_time.dt.date.iloc[0])}")

alt.Chart(...)

# Plot daytime/nighttime-averaged wind speed profiles over the season

In [11]:
src = dl_df.query("x_offset == -250").dropna()
def categorize_daytime(hr):
    if hr in [9,10,11,12,13,14,15]:
        return 'day'
    elif hr in [19,20,21,22,23,0,1,2,3,4,5]:
        return 'night'
    else:
        return np.nan
src.loc[:, ['daytime_category']] = src.scan_time.dt.hour.apply(categorize_daytime)
src = src.dropna()
src['date'] = src['scan_time'].dt.date
src = src.groupby(['daytime_category', 'date', 'z_binned'])[
    ['mean', 'median', 'std']
].agg(
    {'mean': 'mean', 'median': 'median', 'std': 'mean'}
).reset_index()

In [12]:
# Add upper bounds to z and to time for a hovmoller diagram
upper_bound_to_lower_bound = dict(zip(
    sorted(src.z_binned.unique()),
    [0] + sorted(src.z_binned.unique())
))
src['z_high'] = src['z_binned']
src['z_low'] = src['z_high'].apply(upper_bound_to_lower_bound.get)
src['date_high'] = src['date'] + dt.timedelta(hours=24)

In [13]:
night_chart = alt.Chart(
    src.query("daytime_category == 'night'").query("z_binned < 600")
).mark_rect().encode(
    alt.X("date:T").title("date"),
    alt.X2("date_high:T"),
    alt.Y("z_low:Q").title("Height (m, agl)"),
    alt.Y2("z_high:Q"),
    alt.Color("median:Q").scale(scheme='purpleorange', domain=[-6,6], clamp=True).title(["Downvalley", "windspeed (m/s)"]),
    tooltip = 'date'
).properties(
    width = 500, 
    height = 166.66,
    title = "Night time winds (1900 - 0500)"
)

day_chart = alt.Chart(
    src.query("daytime_category == 'day'").query("z_binned < 600")
).mark_rect().encode(
    alt.X("date:T").title("date"),
    alt.X2("date_high:T"),
    alt.Y("z_low:Q").title("Height (m, agl)"),
    alt.Y2("z_high:Q"),
    alt.Color("median:Q").scale(scheme='purpleorange', domain=[-6,6], clamp=True).title(["Downvalley", "windspeed (m/s)"]),
    tooltip = 'date'
).properties(
    width = 500, 
    height = 166.66,
    title = "Day time winds (0900 - 1500)"
)

(night_chart & day_chart).configure_legend(gradientThickness=20, gradientLength = 150)

alt.VConcatChart(...)

# Identify strong/weak synoptics based on 500 mb wind speeds (from radiosonde data)

In [14]:
synoptic_windspeed_daily = synopticwinds_df.groupby(
    [synopticwinds_df.time.dt.date, 'pressure']
)[['wspd']].mean().reset_index().query("pressure == 500")
low_synoptic_dates = synoptic_windspeed_daily.query("wspd < 15").time

NameError: name 'synopticwinds_df' is not defined

Plot the same as above, excluding strong synoptic days

In [15]:

alt.Chart(
    src[src.date.isin(low_synoptic_dates)].query("daytime_category == 'night'").query("z_binned < 600")
).mark_rect().encode(
    alt.X("date:T"),
    alt.X2("date_high:T"),
    alt.Y("z_low:Q"),
    alt.Y2("z_high:Q"),
    alt.Color("median:Q").scale(scheme='purpleorange', domain=[-6,6], clamp=True)
).properties(width = 600, height = 200)

NameError: name 'low_synoptic_dates' is not defined

In [16]:

alt.Chart(
    src[src.date.isin(low_synoptic_dates)].query("daytime_category == 'day'").query("z_binned < 600")
).mark_rect().encode(
    alt.X("date:T"),
    alt.X2("date_high:T"),
    alt.Y("z_low:Q"),
    alt.Y2("z_high:Q"),
    alt.Color("median:Q").scale(scheme='purpleorange', domain=[-6,6], clamp=True)
).properties(width = 600, height = 200)

NameError: name 'low_synoptic_dates' is not defined

# Normalize wind speeds by synoptic wind speeds

In [17]:
synoptic_windspeed_daily.head()

NameError: name 'synoptic_windspeed_daily' is not defined

In [ ]:
src_normalized = src.merge(
    synoptic_windspeed_daily,
    left_on='date',
    right_on = 'time',
    how='left'
)
src_normalized['median_normalized'] = src_normalized['median'] / src_normalized['wspd']

In [ ]:
night_chart = alt.Chart(
    src_normalized.query("daytime_category == 'night'").query("z_binned < 600")
).mark_rect().encode(
    alt.X("date:T").title("date"),
    alt.X2("date_high:T"),
    alt.Y("z_low:Q").title("Height (m, agl)"),
    alt.Y2("z_high:Q"),
    alt.Color("median_normalized:Q").scale(scheme='purpleorange', domain=[-0.5,0.5], clamp=True).title(["Downvalley", "windspeed (m/s)"]),
    tooltip = 'date'
).properties(
    width = 500, 
    height = 166.66,
    title = "Night time winds (1900 - 0500)"
)

day_chart = alt.Chart(
    src_normalized.query("daytime_category == 'day'").query("z_binned < 600")
).mark_rect().encode(
    alt.X("date:T").title("date"),
    alt.X2("date_high:T"),
    alt.Y("z_low:Q").title("Height (m, agl)"),
    alt.Y2("z_high:Q"),
    alt.Color("median_normalized:Q").scale(scheme='purpleorange', domain=[-0.5,0.5], clamp=True).title(["Downvalley", "windspeed (m/s)"]),
    tooltip = 'date'
).properties(
    width = 500, 
    height = 166.66,
    title = "Day time winds (0900 - 1500)"
)

(night_chart & day_chart).configure_legend(gradientThickness=20, gradientLength = 150)

# Plot wind speed hovmoller diagram (using speeds 25 - 175m)

In [18]:
src = dl_df.query("x_offset == -250").dropna()
src = src[
    (src.z_binned >= 25) &
    (src.z_binned <= 175)
]
src = src.dropna()
src['date'] = src['scan_time'].dt.date
src['hour'] = src['scan_time'].dt.hour
src = src.groupby(['date', 'hour'])[
    ['mean', 'median', 'std']
].agg(
    {'mean': 'mean', 'median': 'median', 'std': 'mean'}
).reset_index()

# Add upper bounds to z and to time for a hovmoller diagram
src['date_high'] = src['date'] + dt.timedelta(hours=24)
src['hour_high'] = src['hour'] + 1

src.head()

,date,hour,mean,median,std,date_high,hour_high
0,2023-02-10,16,-0.317581,-0.351027,1.393420,2023-02-11,17
1,2023-02-10,17,2.364419,2.376781,5.680389,2023-02-11,18
2,2023-02-10,18,0.963356,0.491901,4.986455,2023-02-11,19
3,2023-02-10,19,-0.160053,0.508600,3.730716,2023-02-11,20
4,2023-02-10,20,-4.686701,-3.025532,6.108216,2023-02-11,21


In [19]:
alt.Chart(
    src
).mark_rect().encode(
    alt.X("date:T").title("date"),
    alt.X2("date_high:T"),
    alt.Y("hour:Q").title("hour of day"),
    alt.Y2("hour_high:Q"),
    alt.Color("median:Q").scale(scheme='purpleorange', domain=[-6,6], clamp=True).title(["Downvalley", "windspeed (m/s)"]),
    tooltip = 'date'
).properties(
    width = 500, 
    height = 166.66,
    title = "Average wind speed, 25 - 175m"
).display(renderer='svg')

alt.Chart(...)

# Plot monthly-averaged vertical profiles

In [20]:
src = dl_df.query("x_offset == -250").dropna()
src = src.dropna()
src['month'] = src['scan_time'].dt.month
src['hour'] = src['scan_time'].dt.hour

src = src.groupby(['month', 'hour', 'z_binned'])[
    ['mean', 'median', 'std']
].agg(
    {'mean': 'mean', 'median': 'median', 'std': 'mean'}
).reset_index()

alt.Chart(
    src.query("z_binned <= 625").query("month < 6")
).mark_line().encode(
    alt.X("mean:Q").sort('-y'),
    alt.Y("z_binned:Q"),
    alt.Color("month:O").scale(scheme='turbo'),
    alt.Facet("hour", columns=6) 
).properties(width = 125, height = 125)

alt.Chart(...)

In [21]:
src = dl_df.query("x_offset == -250").dropna()
src = src.dropna()
src['month'] = src['scan_time'].dt.month
src['hour'] = src['scan_time'].dt.hour
src['hour_group'] = pd.cut(
    src['hour'], 
    [-1,2.5, 5.5, 8.5, 11.5, 14.5, 17.5, 20.5, 23.5],
    labels=['00-02', '03-05', '06-08', '09-11', '12-14', '15-17', '18-20', '21-23']
)

src = src.groupby(['month', 'hour_group', 'z_binned'])[
    ['mean', 'median', 'std']
].agg(
    {'mean': 'mean', 'median': 'median', 'std': 'mean'}
).reset_index()

alt.Chart(
    src.query("z_binned <= 625").query("month < 6")
).mark_line().encode(
    alt.X("mean:Q").sort('-y').title("Downvalley wind (m/s)"),
    alt.Y("z_binned:Q").title("Height (m, agl)"),
    alt.Color("month:O").scale(scheme='turbo'),
    alt.Facet("hour_group", columns=4).title("hours of day")
).properties(
    width = 125, 
    height = 125,
    title='Mean vertical profiles of valley wind for different parts of the day'
).display(renderer='svg')

/var/folders/x_/2h52bcjx2px15bhmdpdd748h0000gn/T/ipykernel_91346/3937551297.py:11: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  src = src.groupby(['month', 'hour_group', 'z_binned'])[


alt.Chart(...)

In [72]:
src = dl_df[dl_df.x_offset == -250]
src = src[src.scan_time.dt.date.isin([
    dt.date(2023, 4, 16),
    dt.date(2023, 6, 9)
])]
src = src.query("z_binned <= 650")
src['z_binned'] = src['z_binned'].replace({625: 600})
src = src.set_index('scan_time').sort_index()
src = src.groupby([pd.Grouper(freq='120Min'), 'z_binned'])[['median']].median().reset_index()
src['date_str'] = src['scan_time'].dt.date.astype(str)
src['hour'] = src['scan_time'].dt.hour
src['hour'] = src['scan_time'].dt.hour
src['time_category'] = pd.cut(
    src['scan_time'].dt.hour,
    [-0.5, 6.5, 14.5, 23.5],
    labels = ['Morning (0000-0600)', 'Day (0800-1400)', 'Evening (1600-2200)']
)

In [74]:
alt.Chart(src).mark_line().encode(
    alt.X("median:Q").sort('-y').title("Downvalley velocity (m/s)"),
    alt.Y("z_binned:Q").title("Height (m, agl)"),
    alt.Color("hour:O").scale(scheme='sinebow'),
    alt.Row("date_str:N").title(None).header(labelFontSize=16, labelFontWeight='bold'),
    alt.Column("time_category:O").sort(
        ['Morning (0000-0600)', 'Day (0800-1400)', 'Evening (1600-2200)']
    ).title(None).header(labelFontSize=16, labelFontWeight='bold'),
).properties(width = 150, height = 150)

alt.Chart(...)